## 1️⃣ All SQL Example - Ready to copy/paste
- You'll need to store your PAT in a secret
- You'll of course want to update your namespace for the UDF

In [ ]:
%sql
create or replace function users.randy_pitcher.get_serving_endpoint_status(endpoint_name string, auth_token string)
returns string
language python
as $$

from databricks.sdk import WorkspaceClient

w = WorkspaceClient(
    host="https://e2-demo-field-eng.cloud.databricks.com", 
    token=auth_token
)

try:
    endpoint = w.serving_endpoints.get(name=endpoint_name)
    return endpoint.state.ready.value if endpoint.state else "UNKNOWN"
except Exception as e:
    return f"error: {str(e)}"

$$;

In [ ]:
%sql

select 
  users.randy_pitcher.get_serving_endpoint_status(
    'sd-bge-m3-encoder', 
    secret('fieldeng', 'rpw_pat')
  )
  
union all

select 
  users.randy_pitcher.get_serving_endpoint_status(
    'advanced_mlops_churn_ep_mehul', 
    secret('fieldeng', 'rpw_pat')
  );

## 2️⃣ Interactive python to experiment with the approach
- try a few different things here to get an intuition for how it works

### First, let's validate our connection to databricks
- you can just skip this if you run this notebook directly in databricks
- but if you work from cursor or vscode, this is a handy way to connect

In [3]:
# Configure Databricks SDK and Databricks Connect Clients

# Configure Databricks SDK Workspace Client
from databricks.sdk import WorkspaceClient

# Initialize workspace client (will use default authentication flow)
# This will automatically try: notebook-native auth, then environment variables, then .databrickscfg
w = WorkspaceClient()

# Configure Databricks Connect Spark Session
from databricks.connect import DatabricksSession

# Initialize Spark session (will use default authentication flow)
# This will automatically use the same authentication as the workspace client
spark = DatabricksSession.builder.getOrCreate()

# Verify connections
print("✅ Databricks SDK Workspace Client configured")
print(f"   Workspace URL: {w.config.host}")
print(f"   Authentication Type: {w.config.auth_type}")

print("\n✅ Databricks Connect Spark Session configured")
print(f"   Spark Version: {spark.version}")
print("   Spark Session active and ready to use")

# Example usage of both clients
print("\n📊 Example usage:")
print("Workspace Client - List first 3 clusters:")
clusters = w.clusters.list()
for i, cluster in enumerate(clusters):
    if i >= 3:
        break
    print(f"   - {cluster.cluster_name} ({cluster.state})")

print("\nSpark Session - Sample query:")
df = spark.sql("SELECT 'Hello from Databricks Connect!' as message")
df.show()


✅ Databricks SDK Workspace Client configured
   Workspace URL: https://dbc-1f943136-5e42.cloud.databricks.com
   Authentication Type: databricks-cli

✅ Databricks Connect Spark Session configured
   Spark Version: 3.5.2
   Spark Session active and ready to use

📊 Example usage:
Workspace Client - List first 3 clusters:
   - dbr_dev (State.RUNNING)
   - Stupid Single User Cluster (State.RUNNING)
   - dlt-execution-5a0ae2c7-2e2e-440f-a746-2bf5a32ee032 (State.TERMINATED)

Spark Session - Sample query:
+--------------------+
|             message|
+--------------------+
|Hello from Databr...|
+--------------------+



In [4]:
# get all model serving endpoints
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()
endpoints = w.serving_endpoints.list()
display([{"name": ep.name, "state": ep.state.ready.value if ep.state else "UNKNOWN"} for ep in endpoints])

[{'name': 'customer_outreach_ai_builder', 'state': 'READY'},
 {'name': 'kie-d5861e35-endpoint', 'state': 'READY'},
 {'name': 'OpenAI_gpt4', 'state': 'READY'},
 {'name': 'databricks-claude-3-7-sonnet', 'state': 'READY'},
 {'name': 'databricks-claude-sonnet-4', 'state': 'READY'},
 {'name': 'databricks-llama-4-maverick', 'state': 'READY'},
 {'name': 'databricks-gemma-3-12b', 'state': 'READY'},
 {'name': 'databricks-claude-opus-4', 'state': 'READY'},
 {'name': 'databricks-meta-llama-3-1-8b-instruct', 'state': 'READY'},
 {'name': 'databricks-meta-llama-3-3-70b-instruct', 'state': 'READY'},
 {'name': 'databricks-gte-large-en', 'state': 'READY'},
 {'name': 'databricks-meta-llama-3-1-405b-instruct', 'state': 'READY'},
 {'name': 'databricks-bge-large-en', 'state': 'READY'}]

In [5]:
# Get the status of a model serving endpoint
def get_serving_endpoint_status(endpoint_name: str) -> str:
  try:
    w = WorkspaceClient()
    endpoint = w.serving_endpoints.get(name=endpoint_name)
    return endpoint.state.ready.value if endpoint.state else "UNKNOWN"
  except Exception as e:
    return f"ERROR: {str(e)}"

In [8]:
# give the function a try
display(get_serving_endpoint_status('databricks-gemma-3-12b'))

'READY'